In [1]:
import numpy as np
import sys
import matplotlib.pyplot

In [2]:
sys.version_info


sys.version_info(major=2, minor=7, micro=18, releaselevel='final', serial=0)

In [3]:
print "print zuado?"

print zuado?


In [4]:
from pacman import runGames,loadAgent
from pacman import Directions
from util import Queue
import textDisplay
import game
import layout
import random
import graphicsDisplay

qual das 5 direcoes ele toma

- absoluto 
- POV pacman

coisas uteis

- posicao pacman
- quais acoes sao validas
- a posiçao dos fantasma ( distancia aos fantasmas, distancia manhatan)
- a posiçao das comidas ( matriz pode ser grande)
- a posicao das capsulas
- os fantasmas tao assustados ou n

tomar decisao

- 1 equacao pra cada direcao
- emula que vc fez 1 acao legao
    - ve a pontucao = score - p1 * distancia (f1) - p2 * distancia (f2)
- pontua = score -(-1 se assustado) p1 * distancia (f1) - (-1 se assustado) p2 * distancia (f2)



In [5]:
class MyOwnAgent(game.Agent):
    "An agent that does whatever he feels like doing"  

    def getAction(self, state):
        legal = state.getLegalPacmanActions()
        currentPos=state.getPacmanPosition()
        if Directions.STOP in legal: legal.remove(Directions.STOP)
    
        numCapsulesCurrentState=len(state.getCapsules())
        currentScaredGhosts=self.countScaredGhosts(state)
        currentFoodMatrix = state.getFood()
        
        successors = [(state.generateSuccessor(0, action), action) for action in legal]
        scored = [(self.scoreEvaluation(state,numCapsulesCurrentState,currentScaredGhosts,currentPos,currentFoodMatrix), action) for state, action in successors]
        bestScore = max(scored)[0]
        bestActions = [pair[1] for pair in scored if pair[0] == bestScore]
        
        return random.choice(bestActions)
        
    def getLegalSteps(self, pos, walls, visited):
        
        coluna=pos[0]
        linha=pos[1]
        
        lista=[]

        if(coluna+1 <walls.width):
            if(walls[coluna+1][linha]==False and visited[coluna+1][linha]==0):
                lista.append((coluna+1,linha))
        if(coluna-1 >= 0):
            if(walls[coluna-1][linha]==False and visited[coluna-1][linha]==0):
                lista.append((coluna-1,linha))
        if(linha+1 <walls.height ):
            if(walls[coluna][linha+1]==False and visited[coluna][linha+1]==0):
                lista.append((coluna,linha+1))
        if(linha-1 >= 0):
            if(walls[coluna][linha-1]==False and visited[coluna][linha-1]==0):
                lista.append((coluna,linha-1))
        return lista
    
    def findNearestFood(self, state, posIni, posForbidden):
        walls= state.getWalls()
        foods= state.getFood()
        queue = Queue()
        
        
        visited=np.zeros((walls.width,walls.height))
        visited[posForbidden[0]][posForbidden[1]]=1
                
        matHeight=foods.height
        matWidth=foods.width
        
        queue.push((posIni, 1))
        while(not queue.isEmpty()):
            current=queue.pop()
            pos=current[0]
            stepsUntilNow=current[1]
            
            if(foods[pos[0]][pos[1]]==True):
                return stepsUntilNow
            
            visited[pos[0]][pos[1]]=1
            possibles=self.getLegalSteps(pos,walls, visited) # pos = (x,y), walls = matriz true/falso, visitados  [(2,2),(2,1),(2,3)]
            for possibleMove in possibles:
                queue.push( (possibleMove, stepsUntilNow+1))
        return -1
    
    def bfsDistance(self, state, posIni, posForbidden, endPos):
        walls= state.getWalls()
        queue = Queue()
        
        
        visited=np.zeros((walls.width,walls.height))
        visited[posForbidden[0]][posForbidden[1]] = 1
                
        queue.push((posIni, 1))
        while(not queue.isEmpty()):
            current=queue.pop()
            pos=current[0]
            stepsUntilNow=current[1]
            
            if(pos[0] == endPos[0] and pos[1] == endPos[1]):
                return stepsUntilNow
            
            visited[pos[0]][pos[1]]=1
            possibles = self.getLegalSteps(pos,walls, visited) # pos = (x,y), walls = matriz true/falso, visitados  [(2,2),(2,1),(2,3)]
            for possibleMove in possibles:
                queue.push( (possibleMove, stepsUntilNow+1))
        return -1
    
    def ghostDistances(self, ghostPositions, pacmanPosition, state, currentPacmanPos):
        distancias = []
        
        for i in range(len(ghostPositions)):
            dist = self.bfsDistance(state, pacmanPosition, currentPacmanPos, ghostPositions[i])
            distancias.append(dist)
        
        return distancias
    
    def bonusByGhostDistance(self, peso, distancias, state):
        n_ghosts = len(distancias)
        scared_timer = []

        for i in range(n_ghosts):
            scared_timer.append(state.getGhostStates()[i].scaredTimer)
            
        bonus = 0
        for i in range(n_ghosts):
            if(scared_timer[i] == 0):
                bonus += peso*distancias[i]
        return bonus
    
    def capsuleDistanceBonus(self, peso, capsulePositions, pacmanPosition, state, currentPacmanPos):
        bonus = 0
        for capsule in capsulePositions:
            distancia = self.bfsDistance(state, pacmanPosition, currentPacmanPos, capsule)
            bonus -= peso*distancia
        return bonus
    
    def capsuleComsumptionBonus(self, peso, numCapsulesCurrentState, capsulePositions):
        num_capsulas = numCapsulesCurrentState
        num_capsulas_estado_seguinte = len(capsulePositions)
        
        bonus = (num_capsulas - num_capsulas_estado_seguinte) * peso
        
        return bonus
    
    def countScaredGhosts(self, state):
        total=0
        for ghost in state.getGhostStates():
            if(ghost.scaredTimer !=0):
                total+=1
        return total
    
    def scaredGhostBonus(self, state, peso, distancias):
        n_ghosts = self.countScaredGhosts(state)
        scared_timer = []

        for i in range(n_ghosts):
            scared_timer.append(state.getGhostStates()[i].scaredTimer)
            
        bonus = 0
        for i in range(n_ghosts):
            if(scared_timer[i] != 0):
                bonus -= peso*distancias[i]
        return bonus
    
    def eatScaredGhosts(self, state, peso, currentScaredGhosts):
        scaredGhostsInThisState = self.countScaredGhosts(state)
        difScaredGhosts= currentScaredGhosts - scaredGhostsInThisState
        
        bonus=0
        if(difScaredGhosts > 0):
            bonus= peso * difScaredGhosts   
        return bonus
    
    def nearestFoodBonus(self, state, peso, pacmanPosition,currentPacmanPos):
        nearestFoodDistance= self.findNearestFood(state, pacmanPosition,currentPacmanPos)
        bonus = -1* peso * nearestFoodDistance      
        
        return bonus
    
    def eatFoodBonus(self, peso, currentFoodMatrix, foodMatrix):
        diff = currentFoodMatrix.count(True) - foodMatrix.count(True)
        bonus = peso*diff
        
        return bonus
    def scoreEvaluation(self, state, numCapsulesCurrentState, currentScaredGhosts, currentPacmanPos, currentFoodMatrix):
        
        p1= 1.2
        p2= 2.1
        p3= 150
        p4= 2.5
        p5 = 10
        p6 = 2
        p7 = 50
        score = state.getScore()
        
        ghostPositions = state.getGhostPositions() 
        pacmanPosition = state.getPacmanPosition()
        capsulePositions = state.getCapsules() 
        foodMatrix = state.getFood()
        
        distancias_fantasmas = self.ghostDistances(ghostPositions, pacmanPosition, state, currentPacmanPos)
        bonus_distancia_fantasmas = self.bonusByGhostDistance(p1, distancias_fantasmas, state)
        
        bonus_distancia_capsula = self.capsuleDistanceBonus(p2, capsulePositions, pacmanPosition, state, currentPacmanPos)
        bonus_consumir_capsula = self.capsuleComsumptionBonus(p3, numCapsulesCurrentState, capsulePositions) 

        bonus_distancia_fantasmas_assustados = self.scaredGhostBonus(state, p4, distancias_fantasmas)
        bonus_comer_fantasmas = self.eatScaredGhosts(state, p5, currentScaredGhosts)
                
        bonus_comida_proxima =  self.nearestFoodBonus(state, p6, pacmanPosition,currentPacmanPos)              
        bonus_comer_comida = self.eatFoodBonus(p7, currentFoodMatrix, foodMatrix)  
                
        formula = score + bonus_distancia_fantasmas + bonus_distancia_capsula + bonus_consumir_capsula + bonus_distancia_fantasmas_assustados + bonus_comer_fantasmas + bonus_comida_proxima + bonus_comer_comida
        return formula

In [6]:
props={}
pacman = MyOwnAgent()
ghostType = loadAgent('RandomGhost', True)
textDisplay.SLEEP_TIME = 0
props['layout'] = layout.getLayout( 'mediumClassic' )
props['pacman'] = pacman
props['ghosts'] = [ghostType( i+1 ) for i in range( 2 )]
#props['ghosts'] = []
# props['display'] = textDisplay.PacmanGraphics()
#props['display'] = textDisplay.NullGraphics()
props['display'] = graphicsDisplay.PacmanGraphics(1, frameTime = 0.10)
props['numGames'] = 5
props['record'] = False
props['catchExceptions'] = False
props['timeout'] = 10

In [ ]:
res=runGames(**props)

Pacman emerges victorious! Score: 1859


In [ ]:
runGames(**props)

In [ ]:
res[0].state.getScore()

In [ ]:
print(res[0].agentOutput[0].getvalue())

In [ ]:
dir(res[0].agentOutput[0])

In [ ]:
#  pontuacao = 100
#  x O         C = 50
#              C = 50
#              C = -50*30 = -1500
#              C = -50*30 = -1500